In [ ]:
# === CLUSTERING SENSITIVITY ANALYSIS ===
# Validated with: Python 3.9+, numpy, pandas, scikit-learn, matplotlib

import os
import numpy as np
import pandas as pd
from itertools import product
from typing import List, Dict, Tuple

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# -------------------------
# CONFIG — EDIT THESE
# -------------------------
DATA_CSV = "data/clustering_data_combined.parquet"   # <-- path to your table of ~low-variance predictions
OUTDIR   = "cluster_sensitivity_outputs"
os.makedirs(OUTDIR, exist_ok=True)

# Columns used to cluster (e.g., 23 component loadings). You can include properties too if desired.
# Example for 23-component loadings:
FEATURE_COLS = [
    "LAP","MMT","CMC","CNF","SLK","GEL","CHS","AGR","ALG","PUL","CAR","STA",
    "FFA","PEC","ZIN","GLU","GLY","XYL","SRB","PHA","LAC","LEV","SUA"
]

# If you prefer clustering in property space (or combined), set columns accordingly, e.g.:
# FEATURE_COLS = ["sigma_u_pred","SED_pred","Tvis_pred"]  # example property-space clustering

PRED_VAR_COL = "Uncertainty"  # prediction variance column
PRED_VAR_MAX = 0.15        # keep only predictions below this variance threshold

# (Optional) multi-property gates — set to None to skip
PROPERTY_GATES = {
    # "sigma_u_pred": (100, None),  # >=100 MPa
    # "Tvis_pred":    (90,  None),  # >=90 %
    # "SED_pred":     (15,  None),  # >=15 MJ/m^3 (or cm^3 if that’s your unit)
}

# Baseline DBSCAN params (these are the values you used in the paper)
BASELINE = {"eps": 0.15, "min_samples": 20}

# Grids to test for sensitivity
EPS_GRID          = np.round(np.linspace(0.05, 0.30, 11), 3)   # 0.05 ... 0.30
MIN_SAMPLES_GRID  = [5, 10, 15, 20, 25, 30]

# Whether to standardize features (recommended)
STANDARDIZE = True

# Random seed for reproducibility (only used for visualization reductions)
RANDOM_STATE = 42


# -------------------------
# LOAD & FILTER DATA
# -------------------------
def load_and_filter(csv_path: str,
                    feature_cols: List[str],
                    pred_var_col: str,
                    pred_var_max: float,
                    property_gates: Dict[str, Tuple[float,float]] = None) -> pd.DataFrame:
    df = pd.read_parquet(csv_path).sample(frac=0.2)
    df[pred_var_col] = df[pred_var_col]/100
    missing = [c for c in feature_cols + [pred_var_col] if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in CSV: {missing}")

    # Filter by prediction variance
    df = df[df[pred_var_col] <= pred_var_max].copy()

    # Optional property gates
    if property_gates:
        for col, (lo, hi) in property_gates.items():
            if col not in df.columns:
                raise ValueError(f"Gate column '{col}' not found in data.")
            if lo is not None:
                df = df[df[col] >= lo]
            if hi is not None:
                df = df[df[col] <= hi]
        df = df.copy()

    if feature_cols:
        X = df[feature_cols].values
    else:
        # if no feature list given, assume all numeric except known non-features
        non_feature = {pred_var_col}
        if property_gates:
            non_feature |= set(property_gates.keys())
        feature_cols_auto = [c for c in df.columns if (df[c].dtype != "O" and c not in non_feature)]
        print(f"[INFO] FEATURE_COLS not provided. Auto-detected numeric columns (excl. gates/variance): {feature_cols_auto}")
        X = df[feature_cols_auto].values

    return df, X, (feature_cols if feature_cols else feature_cols_auto)


# -------------------------
# CLUSTERING HELPERS
# -------------------------
def run_dbscan(X: np.ndarray, eps: float, min_samples: int) -> np.ndarray:
    cl = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
    return cl.fit_predict(X)

def clustering_quality(X: np.ndarray, labels: np.ndarray) -> Dict[str, float]:
    # Exclude noise (-1) for silhouette
    mask = labels != -1
    quality = {
        "n_points": int(X.shape[0]),
        "n_noise": int(np.sum(~mask)),
        "noise_frac": float(np.mean(~mask)),
        "n_clusters": int(len(set(labels)) - (1 if -1 in labels else 0)),
        "silhouette": np.nan
    }
    # Silhouette defined only if >=2 clusters and at least 1 non-noise point per cluster
    if np.sum(mask) > 1 and quality["n_clusters"] >= 2:
        try:
            quality["silhouette"] = float(silhouette_score(X[mask], labels[mask]))
        except Exception:
            pass
    return quality

def compare_partitions(y_ref: np.ndarray, y_alt: np.ndarray) -> Dict[str, float]:
    # Stability measures (treat noise as its own label)
    return {
        "ARI_vs_baseline": float(adjusted_rand_score(y_ref, y_alt)),
        "NMI_vs_baseline": float(normalized_mutual_info_score(y_ref, y_alt))
    }


# -------------------------
# MAIN ANALYSIS
# -------------------------
def main():
    df, X_raw, used_feats = load_and_filter(
        DATA_CSV, FEATURE_COLS, PRED_VAR_COL, PRED_VAR_MAX, PROPERTY_GATES
    )
    print(f"[INFO] Data after filtering: {df.shape[0]} rows")
    print(f"[INFO] Using {len(used_feats)} feature columns: {used_feats}")

    # Standardize (recommended for distance-based clustering)
    if STANDARDIZE:
        scaler = StandardScaler()
        X = scaler.fit_transform(X_raw)
    else:
        X = X_raw

    # Baseline DBSCAN
    base_labels = run_dbscan(X, eps=BASELINE["eps"], min_samples=BASELINE["min_samples"])
    base_stats  = clustering_quality(X, base_labels)
    base_stats.update({"eps": BASELINE["eps"], "min_samples": BASELINE["min_samples"], "algorithm": "DBSCAN"})
    print("[BASELINE] ", base_stats)

    # Grid search DBSCAN sensitivity
    rows = []
    for eps, ms in product(EPS_GRID, MIN_SAMPLES_GRID):
        labels = run_dbscan(X, eps=eps, min_samples=ms)
        stats  = clustering_quality(X, labels)
        stab   = compare_partitions(base_labels, labels)
        stats.update({"eps": float(eps), "min_samples": int(ms), "algorithm": "DBSCAN"})
        stats.update(stab)
        rows.append(stats)

    sens_df = pd.DataFrame(rows).sort_values(["eps", "min_samples"]).reset_index(drop=True)
    sens_csv = os.path.join(OUTDIR, "dbscan_sensitivity_summary.csv")
    sens_df.to_csv(sens_csv, index=False)
    print(f"[SAVE] DBSCAN sensitivity table -> {sens_csv}")

    # -------------------------
    # PLOTS (simple, matplotlib, 1 chart per fig; no explicit colors)
    # -------------------------

    # 1) Heatmap-like scatter: ARI vs baseline as a function of eps & min_samples
    fig1 = plt.figure(figsize=(7,5))
    pivot_ari = sens_df.pivot(index="min_samples", columns="eps", values="ARI_vs_baseline")
    plt.imshow(pivot_ari.values, aspect="auto", origin="lower", extent=[
        pivot_ari.columns.min(), pivot_ari.columns.max(), pivot_ari.index.min(), pivot_ari.index.max()
    ])
    plt.colorbar()
    plt.xlabel("eps")
    plt.ylabel("min_samples")
    plt.title("DBSCAN stability (ARI vs. baseline)")
    plt.tight_layout()
    fig1_path = os.path.join(OUTDIR, "dbscan_ari_heatmap.png")
    plt.savefig(fig1_path, dpi=200)
    plt.close()
    print(f"[SAVE] {fig1_path}")

    # 2) Line plot: noise fraction vs eps for each min_samples
    fig2 = plt.figure(figsize=(7,5))
    for ms in MIN_SAMPLES_GRID:
        sub = sens_df[sens_df["min_samples"]==ms]
        plt.plot(sub["eps"], sub["noise_frac"], marker="o", label=f"min_samples={ms}")
    plt.xlabel("eps")
    plt.ylabel("noise fraction")
    plt.title("DBSCAN noise fraction across eps")
    plt.legend()
    plt.tight_layout()
    fig2_path = os.path.join(OUTDIR, "dbscan_noise_fraction.png")
    plt.savefig(fig2_path, dpi=200)
    plt.close()
    print(f"[SAVE] {fig2_path}")

    # 3) Line plot: number of clusters vs eps
    fig3 = plt.figure(figsize=(7,5))
    for ms in MIN_SAMPLES_GRID:
        sub = sens_df[sens_df["min_samples"]==ms]
        plt.plot(sub["eps"], sub["n_clusters"], marker="o", label=f"min_samples={ms}")
    plt.xlabel("eps")
    plt.ylabel("# clusters")
    plt.title("DBSCAN number of clusters across eps")
    plt.legend()
    plt.tight_layout()
    fig3_path = os.path.join(OUTDIR, "dbscan_num_clusters.png")
    plt.savefig(fig3_path, dpi=200)
    plt.close()
    print(f"[SAVE] {fig3_path}")

    # 4) Optional: a simple 2D PCA scatter with baseline labels (quick visual sanity check)
    try:
        pca = PCA(n_components=2, random_state=RANDOM_STATE)
        X2 = pca.fit_transform(X)
        fig5 = plt.figure(figsize=(6,5))
        # Noise first
        noise_mask = base_labels == -1
        plt.scatter(X2[noise_mask,0], X2[noise_mask,1], s=5, alpha=0.5, label="noise")
        # Clusters
        for lab in sorted(set(base_labels) - {-1}):
            m = base_labels == lab
            plt.scatter(X2[m,0], X2[m,1], s=5, alpha=0.8, label=f"cluster {lab}")
        plt.title(f"Baseline DBSCAN (eps={BASELINE['eps']}, min_samples={BASELINE['min_samples']})")
        plt.xlabel("PC1")
        plt.ylabel("PC2")
        plt.legend(markerscale=3)
        plt.tight_layout()
        fig4_path = os.path.join(OUTDIR, "baseline_pca_scatter.png")
        plt.savefig(fig4_path, dpi=200)
        plt.close()
        print(f"[SAVE] {fig4_path}")
    except Exception as e:
        print(f"[WARN] PCA scatter skipped: {e}")

    # -------------------------
    # PRINT A QUICK TEXT SUMMARY
    # -------------------------
    def short_stats(df, eps, ms):
        row = df[(df["eps"]==eps) & (df["min_samples"]==ms)].head(1)
        if row.empty: return None
        r = row.iloc[0].to_dict()
        return {
            "n_clusters": r["n_clusters"],
            "noise_frac": r["noise_frac"],
            "silhouette": r["silhouette"],
            "ARI_vs_baseline": r["ARI_vs_baseline"],
            "NMI_vs_baseline": r["NMI_vs_baseline"]
        }

    print("\n=== QUICK CHECKPOINTS ===")
    for (e, m) in [(0.10, 20), (0.15, 20), (0.20, 20)]:
        s = short_stats(sens_df, e, m)
        if s:
            print(f"eps={e}, min_samples={m} -> {s}")
    print("Done.")



In [8]:
main()

[INFO] Data after filtering: 95597 rows
[INFO] Using 23 feature columns: ['LAP', 'MMT', 'CMC', 'CNF', 'SLK', 'GEL', 'CHS', 'AGR', 'ALG', 'PUL', 'CAR', 'STA', 'FFA', 'PEC', 'ZIN', 'GLU', 'GLY', 'XYL', 'SRB', 'PHA', 'LAC', 'LEV', 'SUA']
[BASELINE]  {'n_points': 95597, 'n_noise': 95549, 'noise_frac': 0.9994978921932697, 'n_clusters': 2, 'silhouette': 0.4452048075870825, 'eps': 0.15, 'min_samples': 20, 'algorithm': 'DBSCAN'}
[SAVE] DBSCAN sensitivity table -> cluster_sensitivity_outputs/dbscan_sensitivity_summary.csv


KeyboardInterrupt: 

In [ ]:
# Run the streamlined sensitivity analysis (OPTICS removed for speed)
print("🚀 Starting streamlined DBSCAN sensitivity analysis...")
main()
